<a href="https://colab.research.google.com/github/EnaJeong/LogInAndOut-log-severity-level-analysis/blob/kangin/EDA/EDA_ki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import pandas as pd
# from dask import dataframe
import re
import numpy as np
import seaborn as sbn

from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

## Load DATA

In [ ]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
%%time

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
# validation_df = pd.read_csv('./Dacon_dataset/validation_sample.csv')
# test_df = pd.read_csv('./Dacon_dataset/test.csv')
# submission_df = pd.read_csv('./Dacon_dataset/sample_submission.csv')

CPU times: user 2.24 s, sys: 420 ms, total: 2.66 s
Wall time: 4.41 s


## Raw Data Anlysis

In [ ]:
train_df.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
1,1,0,"Feb 8 16:21:00 localhost logstash: [2021-02-08T16:21:00,548][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[wazuh-alerts-audit-3.x-2021.16.08][3] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-3.x-2021.16.08][3]] containing [25] requests]""})"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
4,4,1,"type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm=""date"" exe=""/usr/bin/date"" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603094402.016:52981): cwd=""/root"" type=PATH msg=audit(1603094402.016:52981): item=0 name=""/etc/ld.so.cache"" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564"


In [ ]:
train_df.value_counts('level')

level
0    334065
1    132517
3    4141  
5    2219  
2    12    
4    10    
6    8     
dtype: int64

In [ ]:
train_df.value_counts('level',normalize=True)

level
0    0.706310
1    0.280179
3    0.008755
5    0.004692
2    0.000025
4    0.000021
6    0.000017
dtype: float64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472972 entries, 0 to 472971
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        472972 non-null  int64 
 1   level     472972 non-null  int64 
 2   full_log  472972 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.8+ MB


## value가 적은 단어들의 반복되는 특징 

## Level 6

In [ ]:
lv_6 = train_df.loc[train_df.level==6]

In [ ]:
lv_6 #8개

,id,level,full_log
87783,87783,6,Jan 30 08:23:17 localhost sshd[18415]: Bad protocol version identification '\003' from 78.128.113.18 port 1073
170386,170386,6,Feb 8 16:16:47 localhost kernel: device virbr0-nic entered promiscuous mode
213304,213304,6,Mar 8 15:29:30 localhost kernel: device enp2s0 entered promiscuous mode
221847,221847,6,"Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001"
223644,223644,6,Jan 22 15:18:49 localhost sshd[19015]: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 61153 ssh2 [preauth]
361823,361823,6,Feb 2 17:22:58 localhost kernel: device virbr0-nic entered promiscuous mode
406796,406796,6,Feb 4 09:59:33 localhost kernel: device virbr0-nic entered promiscuous mode
429489,429489,6,Feb 4 09:59:43 localhost kernel: device enp2s0 entered promiscuous mode


In [ ]:
train_df[train_df['full_log'].str.contains('entered promiscuous mode')].value_counts('level') 

level
6    5
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Bad protocol version identification')].value_counts('level') 

level
6    1
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('maximum authentication attempts exceeded')].value_counts('level')

level
6    1
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('localhost useradd')].value_counts('level')

level
6    1
dtype: int64

## Level4

In [ ]:
lv_4 = train_df.loc[train_df.level==4]

In [ ]:
lv_4 #10개

,id,level,full_log
64919,64919,4,Nov 29 05:44:21 localhost sshd[6008]: Did not receive identification string from 211.253.243.66 port 57487
85354,85354,4,Nov 29 22:16:32 sv260 sshd[39585]: Did not receive identification string from 211.253.243.66
123069,123069,4,Nov 3 10:38:05 localhost sshd[76373]: Did not receive identification string from 192.168.0.195 port 3819
167164,167164,4,Nov 21 01:02:56 sv260 sshd[9196]: Did not receive identification string from 211.253.243.66
184483,184483,4,Nov 27 08:32:12 sv260 sshd[3110]: Did not receive identification string from 192.168.0.195
230131,230131,4,Nov 19 01:03:09 sv260 sshd[13841]: Did not receive identification string from 192.168.0.195
321526,321526,4,Nov 12 04:27:24 localhost sshd[98418]: Did not receive identification string from 61.41.101.142 port 8975
348055,348055,4,Jan 27 20:20:54 localhost sshd[5025]: Did not receive identification string from 35.184.222.44 port 46232
348684,348684,4,Nov 3 06:03:17 localhost sshd[4346]: Did not receive identification string from 192.168.0.181 port 82066
369842,369842,4,Nov 8 20:25:42 sv260 sshd[37597]: Did not receive identification string from 192.168.0.181


In [ ]:
train_df[train_df['full_log'].str.contains('Did not receive identification string from')].value_counts('level')

level
4    10
dtype: int64

## Level2

In [ ]:
lv_2 = train_df.loc[train_df.level==2]

NameError: ignored

In [ ]:
lv_2 #12개

In [ ]:
train_df[train_df['full_log'].str.contains('The average number of logs between')].value_counts('level')

In [ ]:
train_df[train_df['full_log'].str.contains('XGET')].value_counts('level') #localhost sudo

## Level5

In [ ]:
lv_5 = train_df.loc[train_df.level==5]

In [ ]:
lv_5.head(50) #2219

,id,level,full_log
590,590,5,ossec: output: 'netstat listening ports':\ntcp 0.0.0.0:22 0.0.0.0:* 1428/sshd\ntcp6 :::22 :::* 1428/sshd\ntcp 127.0.0.1:25 0.0.0.0:* 2197/master\ntcp6 ::1:25 :::* 2197/master\ntcp 192.168.122.1:53 0.0.0.0:* 2130/dnsmasq\nudp 192.168.122.1:53 0.0.0.0:* 2130/dnsmasq\nudp 0.0.0.0:67 0.0.0.0:* 2130/dnsmasq\ntcp 0.0.0.0:300 0.0.0.0:* 27197/\nudp 127.0.0.1:323 0.0.0.0:* 890/chronyd\nudp6 ::1:323 :::* 890/chronyd\ntcp6 :::443 :::* 1465/httpd\nudp 0.0.0.0:514 0.0.0.0:* 27133/ossec-remoted\ntcp 127.0.0.1:631 0.0.0.0:* 1432/cupsd\ntcp6 ::1:631 :::* 1432/cupsd\ntcp 0.0.0.0:1514 0.0.0.0:* 27132/ossec-remoted\ntcp 0.0.0.0:1515 0.0.0.0:* 27092/ossec-authd\ntcp6 :::3306 :::* 1670/mysqld\nudp 0.0.0.0:5353 0.0.0.0:* 845/avahi-daemon\ntcp 0.0.0.0:5601 0.0.0.0:* 5942/node\ntcp6 127.0.0.1:9200 :::* 15254/java\ntcp6 ::1:9200 :::* 15254/java\ntcp6 127.0.0.1:9300 :::* 15254/java\ntcp6 ::1:9300 :::* 15254/java\ntcp6 127.0.0.1:9600 :::* 6115/java\ntcp6 :::55000 :::* 5890/node\nudp 0.0.0.0:55872 0.0.0.0:* 845/avahi-daemon
621,621,5,ossec: output: 'netstat listening ports':\ntcp 211.253.243.71.49391 61.41.101.142.1514\ntcp 211.253.243.71.49177 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49172\ntcp 211.253.243.71.49172 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49177\ntcp 211.253.243.71.6000 211.253.243.71.49170\ntcp 211.253.243.71.49170 211.253.243.71.6000\ntcp 211.253.243.71.51932 211.253.243.71.1712\nudp 127.0.0.1.49164 127.0.0.1.49164
794,794,5,File '/var/www/html_update/management/hashconf.txt' is owned by root and has written permissions to anyone.
870,870,5,ossec: output: 'netstat listening ports':\ntcp 211.253.243.71.49391 61.41.101.142.1514\ntcp 211.253.243.71.49177 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49172\ntcp 211.253.243.71.49172 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49177\ntcp 211.253.243.71.6000 211.253.243.71.49170\ntcp 211.253.243.71.49170 211.253.243.71.6000\ntcp 211.253.243.71.52348 211.253.243.71.1712\nudp 127.0.0.1.49164 127.0.0.1.49164
1076,1076,5,"level : 10, log : fru-name#fru-slot - command"
1853,1853,5,ossec: output: 'netstat listening ports':\ntcp *.* *.* CLOSED\ntcp *.111 *.* LISTEN\ntcp *.14248 *.* LISTEN\ntcp *.16191 *.* LISTEN\ntcp *.21 *.* LISTEN\ntcp *.23 *.* LISTEN\ntcp *.32771 *.* LISTEN\ntcp *.32772 *.* LISTEN\ntcp *.32777 *.* LISTEN\ntcp *.32784 *.* LISTEN\ntcp *.32787 *.* LISTEN\ntcp *.427 *.* LISTEN\ntcp *.512 *.* LISTEN\ntcp *.513 *.* LISTEN\ntcp *.514 *.* LISTEN\ntcp *.5335 *.* LISTEN\ntcp *.5336 *.* LISTEN\ntcp *.5988 *.* LISTEN\ntcp *.5989 *.* LISTEN\ntcp *.6181 *.* LISTEN\ntcp *.657 *.* LISTEN\ntcp *.6987 *.* LISTEN\ntcp *.6988 *.* LISTEN\ntcp *.9510 *.* LISTEN\ntcp4 *.* *.* CLOSED\ntcp4 *.13 *.* LISTEN\ntcp4 *.1334 *.* LISTEN\ntcp4 *.22 *.* LISTEN\ntcp4 *.25 *.* LISTEN\ntcp4 *.32768 *.* LISTEN\ntcp4 *.32769 *.* LISTEN\ntcp4 *.32805 *.* LISTEN\ntcp4 *.32806 *.* LISTEN\ntcp4 *.32808 *.* LISTEN\ntcp4 *.32810 *.* LISTEN\ntcp4 *.32828 *.* LISTEN\ntcp4 *.37 *.* LISTEN\ntcp4 *.4000 *.* LISTEN\ntcp4 *.45650 *.* LISTEN\ntcp4 *.6000 *.* LISTEN\ntcp4 *.6112 *.* LISTEN\ntcp4 127.0.0.1.9514 *.* LISTEN\ntcp4 127.0.0.1.9515 *.* LISTEN\ntcp4 211.253.243.72.32769 211.253.243.72.784 ESTABLISHED\ntcp4 211.253.243.72.32805 211.253.243.72.659 ESTABLISHED\ntcp4 211.253.243.72.32805 211.253.243.72.703 ESTABLISHED\ntcp4 211.253.243.72.32805 211.253.243.72.755 ESTABLISHED\ntcp4 211.253.243.72.32805 211.253.243.72.778 ESTABLISHED\ntcp4 211.253.243.72.32805 211.253.243.72.904 ESTABLISHED\ntcp4 211.253.243.72.32806 211.253.243.72.32807 ESTABLISHED\ntcp4 211.253.243.72.32807 211.253.243.72.32806 ESTABLISHED\ntcp4 211.253.243.72.32808 211.253.243.72.32809 ESTABLISHED\ntcp4 211.253.243.72.32809 211.253.243.72.32808 ESTABLISHED\ntcp4 211.253.243.72.32810 211.253.243.72.32811 ESTABLISHED\ntcp4 211.253.243.72.32811 211.253.243.72.32810 ESTABLISHED\ntcp4 211.253.243.72.32828 211.253.243.72.32829 ESTABLISHED\ntcp4 211.253.243.72.32829 211.2

In [ ]:
lv_5_1 = lv_5[~lv_5['full_log'].str.contains('netstat listening ports')]

In [ ]:
lv_5_1.head(50)

,id,level,full_log
794,794,5,File '/var/www/html_update/management/hashconf.txt' is owned by root and has written permissions to anyone.
1076,1076,5,"level : 10, log : fru-name#fru-slot - command"
2858,2858,5,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\mouhid' checksum changed.\nOld md5sum was: '6bf8548555a7ab4a34b83436539b6495'\nNew md5sum is : '85c899e225e74c712f8e2d2ef32924c0'\nOld sha1sum was: 'c34db755b2af9472d44d2a51e87699991fee9b8e'\nNew sha1sum is : 'efdfe108fb55c6a564a3c1821af6b388f9808223'\n
4161,4161,5,"File '/etc/dfs/sharetab' checksum changed.\nOld modification time was: 'Mon Nov 16 13:01:41 2020', now it is 'Thu Nov 19 08:55:16 2020'\n"
4704,4704,5,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\SmsRouter\State\Registration\Map' checksum changed.\nOld md5sum was: '664191fb850026c06ea88ed971daa7fd'\nNew md5sum is : '9b126e6218876ca57284259416fd8e6e'\nOld sha1sum was: '23f3ae9ce553344e9361e5187e81946162119151'\nNew sha1sum is : '52143ce22a06f4d78e7ae83193b02737902169df'\n
4728,4728,5,Trojaned version of file '/bin/netstat' detected. Signature used: 'bash|^/bin/sh|/dev/[^aik]|/prof|grep|addr\.h' (Generic).
4996,4996,5,File '/var/www/html/management/hashconf.txt' is owned by root and has written permissions to anyone.
5651,5651,5,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\.NET CLR Data\Performance' checksum changed.\nOld md5sum was: 'bee03237ea803f91e468ce5404efa622'\nNew md5sum is : '2f43dfc7a4f84a246bcfda248e5ab44f'\nOld sha1sum was: 'f591a0f3910caa91e69c116fe9a7091a01c27560'\nNew sha1sum is : 'e8c0484ba1276d30bd9f673e47502e2c0eca65e1'\n
5658,5658,5,"oscap: msg: ""xccdf-result"", scan-id: ""0001603070557"", content: ""ssg-centos-7-ds.xml"", title: ""Ensure auditd Collects Information on the Use of Privileged Commands"", id: ""xccdf_org.ssgproject.content_rule_audit_rules_privileged_commands"", result: ""fail"", severity: ""medium"", description: ""At a minimum, the audit system should collect the execution of privileged commands for all users and root. To find the relevant setuid / setgid programs, run the following command for each local partition PART: $ sudo find PART -xdev -type f -perm -4000 -o -type f -perm -2000 2>/dev/null If the auditd daemon is configured to use the augenrules program to read audit rules during daemon startup (the default), add a line of the following form to a file with suffix .rules in the directory /etc/audit/rules.d for each setuid / setgid program on the system, replacing the SETUID_PROG_PATH part with the full path of that setuid / setgid program in the list: -a always,exit -F path=SETUID_PROG_PATH -F perm=x -F auid>=1000 -F auid!=4294967295 -F key=privileged If the auditd daemon is configured to use the auditctl utility to read audit rules during daemon startup, add a line of the following form to /etc/audit/audit.rules for each setuid / setgid program on the system, replacing the SETUID_PROG_PATH part with the full path of that setuid / setgid program in the list: -a always,exit -F path=SETUID_PROG_PATH -F perm=x -F auid>=1000 -F auid!=4294967295 -F key=privileged"", rationale: ""Misuse of privileged functions, either intentionally or unintentionally by authorized users, or by unauthorized external entities that have compromised system accounts, is a serious and ongoing concern and can have significant adverse impacts on organizations. Auditing the use of privileged functions is one way to detect such misuse and identify the risk from insider and advanced persistent threast. Privileged programs are subject to escalation-of-privilege attacks, which attempt to subvert their normal role of providing some necessary but limited capability. As such, motivation exists to monitor these programs for unusual activity."" references: ""AC-17(7) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-1(b) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-2(a) (http://nvlpubs.nist.gov/nistpubs/Specia

In [ ]:
train_df[train_df['full_log'].str.contains('netstat listening ports')].value_counts('level')

level
5    1289
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('checksum changed')].value_counts('level')

level
5    397
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('is owned by root and has written permissions to anyone')].value_counts('level')

level
5    244
0    6  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('NTFS Alternate data stream found')].value_counts('level')

level
5    25
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('localhost yum')].value_counts('level')

level
5    16
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Trojaned version of file')].value_counts('level')

level
5    15
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('oscap')].value_counts('level')

level
3    198
5    168
1    12 
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 10')].value_counts('level')

level
0    27
1    13
5    3 
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 8')].value_counts('level')

level
1    1
5    1
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 5')].value_counts('level')

level
0    859
1    422
5    41 
3    4  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 3')].value_counts('level')

level
0    593
1    282
5    18 
3    7  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 2')].value_counts('level')

level
0    18
1    6 
5    2 
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('level : 1')].value_counts('level')

level
0    27
1    13
5    3 
dtype: int64

In [ ]:
# train_df[train_df['full_log'].str.contains('File \'HKEY_LOCAL_MACHINE')].value_counts('level')

level
5    303
3    168
dtype: int64

In [ ]:
#lv_5_2 = lv_5_2[~lv_5_2['full_log'].str.contains('oscap')]

In [ ]:
#lv_5_2 = lv_5_2.loc[lv_5_2.level==5]

In [ ]:
#lv_5_2.head(50)

,id,level,full_log
1076,1076,5,"level : 10, log : fru-name#fru-slot - command"
4728,4728,5,Trojaned version of file '/bin/netstat' detected. Signature used: 'bash|^/bin/sh|/dev/[^aik]|/prof|grep|addr\.h' (Generic).
6617,6617,5,Jan 15 15:55:41 localhost yum[26878]: Installed: 1:openssl-devel-1.0.2k-21.el7_9.x86_64
6970,6970,5,"level : 3, log : function-name: operation ioctl failure for group group-id at address 0xaddress (errno error-code)"
26041,26041,5,NTFS Alternate data stream found: 'C:\Program Files/Sublime Text 3:Win32App_1'. Possible hidden content.
34838,34838,5,NTFS Alternate data stream found: 'C:\Program Files/rempl:Win32App_1'. Possible hidden content.
49230,49230,5,NTFS Alternate data stream found: 'C:\Program Files/Sublime Text 3:Win32App_1'. Possible hidden content.
57316,57316,5,"level : 5, log : MPLS LSP lsp-name down on path-type(pathname)"
57641,57641,5,"level : 3, log : executable-name: missing argument for switch: option"
58077,58077,5,Trojaned version of file '/usr/bin/netstat' detected. Signature used: 'bash|^/bin/sh|/dev/[^aik]|/prof|grep|addr\.h' (Generic).


In [ ]:
#lv_5_2.value_counts('level')

level
5    303
3    168
dtype: int64

In [ ]:
#lv_5_2[~lv_5_2['full_log'].str.contains('checksum changed')].value_counts('level')

level
3    168
dtype: int64

## Level 3

In [ ]:
lv_3 = train_df.loc[train_df.level==3] #4141

In [ ]:
lv_3.head(50)

,id,level,full_log
337,337,3,Jan 22 11:37:57 localhost systemd: Unit esild-ml.service entered failed state.
515,515,3,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\BluetoothUserService_4ceeb83\Security' was added.\n
638,638,3,Jan 27 20:31:12 localhost sshd[6677]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=44.222.184.35.bc.googleusercontent.com
1058,1058,3,Jan 22 04:41:11 localhost systemd: Unit esild-ml.service entered failed state.
1560,1560,3,Jan 22 11:26:08 localhost systemd: Unit esild-ml.service entered failed state.
1683,1683,3,Jan 22 12:49:41 localhost systemd: Unit esild-ml.service entered failed state.
1924,1924,3,Jan 22 02:33:45 localhost systemd: Unit esild-ml.service entered failed state.
1955,1955,3,Jan 22 11:03:01 localhost systemd: Unit esild-ml.service entered failed state.
1997,1997,3,File '/usr/bin/elasticdump' was added.\n
2260,2260,3,Jan 22 03:27:55 localhost systemd: Unit esild-ml.service entered failed state.


In [ ]:
lv_3_3 = lv_3.copy()

In [ ]:
lv_3_3 = lv_3_3[lv_3_3['full_log'].str.contains('Failed password for')]

In [ ]:
lv_3_3.head(50)

,id,level,full_log
2570,2570,3,Jan 24 03:03:17 localhost sshd[32346]: Failed password for root from 35.213.176.172 port 34458 ssh2
3868,3868,3,Jan 24 03:05:03 localhost sshd[32611]: Failed password for invalid user acer from 35.213.176.172 port 35938 ssh2
6035,6035,3,Jan 28 22:53:19 localhost sshd[28662]: Failed password for invalid user xmk from 95.111.236.177 port 54904 ssh2
6640,6640,3,Jan 24 02:59:45 localhost sshd[31607]: Failed password for invalid user dedi from 35.213.176.172 port 58854 ssh2
6716,6716,3,Feb 1 14:38:46 localhost sshd[29630]: Failed password for invalid user tea45st1234 from 192.168.0.169 port 45546 ssh2
7023,7023,3,Jan 29 09:07:04 localhost sshd[28757]: Failed password for invalid user test1234 from 192.168.0.169 port 42152 ssh2
7890,7890,3,Jan 27 20:39:19 localhost sshd[7905]: Failed password for invalid user oracle from 35.184.222.44 port 38736 ssh2
8645,8645,3,Jan 27 20:25:28 localhost sshd[5654]: Failed password for invalid user user from 35.184.222.44 port 48122 ssh2
9702,9702,3,Jan 24 02:59:33 localhost sshd[31578]: Failed password for invalid user ubuntu from 35.213.176.172 port 41488 ssh2
10245,10245,3,Jan 28 22:28:21 localhost sshd[26701]: Failed password for root from 95.111.236.177 port 47310 ssh2


In [ ]:
lv_3_2 = lv_3_2[~lv_3_2['full_log'].str.contains('oscap')]

In [ ]:
lv_3_2.head(50)

,id,level,full_log
2570,2570,3,Jan 24 03:03:17 localhost sshd[32346]: Failed password for root from 35.213.176.172 port 34458 ssh2
3868,3868,3,Jan 24 03:05:03 localhost sshd[32611]: Failed password for invalid user acer from 35.213.176.172 port 35938 ssh2
4337,4337,3,Jan 24 03:00:03 localhost unix_chkpwd[31797]: password check failed for user (root)
4841,4841,3,Nov 17 14:29:00 sv260 sshd[6563]: Failed none for invalid user test from 61.41.101.142 port 52401 ssh2
6035,6035,3,Jan 28 22:53:19 localhost sshd[28662]: Failed password for invalid user xmk from 95.111.236.177 port 54904 ssh2
6640,6640,3,Jan 24 02:59:45 localhost sshd[31607]: Failed password for invalid user dedi from 35.213.176.172 port 58854 ssh2
6716,6716,3,Feb 1 14:38:46 localhost sshd[29630]: Failed password for invalid user tea45st1234 from 192.168.0.169 port 45546 ssh2
6805,6805,3,"level : 3, log : Unable to rename file 'source-filename' to 'destination-filename': reason"
7023,7023,3,Jan 29 09:07:04 localhost sshd[28757]: Failed password for invalid user test1234 from 192.168.0.169 port 42152 ssh2
7253,7253,3,Jan 28 22:28:56 localhost unix_chkpwd[26745]: password check failed for user (root)


In [ ]:
train_df[train_df['full_log'].str.contains('localhost systemd')].value_counts('level')

level
3    2210
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Failed password for')].value_counts('level')

level
3    582
0    1  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('authentication failure')].value_counts('level')

level
3    456
1    1  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Invalid user')].value_counts('level')

level
3    336
0    1  
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('was added')].value_counts('level')

level
3    181
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('password check failed for user')].value_counts('level')

level
3    122
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('\(NULL\)')].value_counts('level')

level
3    16
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('NULL')].value_counts('level')

level
0    23
3    16
1    2 
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('WinEvtLog:')].value_counts('level')

level
0    88
3    31
dtype: int64

In [ ]:
lv_3_1 = lv_3_1[~lv_3_1['full_log'].str.contains('WinEvtLog:')]

In [ ]:
lv_3_1.head(50)

,id,level,full_log
6805,6805,3,"level : 3, log : Unable to rename file 'source-filename' to 'destination-filename': reason"
14398,14398,3,"level : 5, log : Dynamic VPN license granted, license limit free, count overdrafted, current-value used"
21121,21121,3,"level : 5, log : Received IPC message with incorrect type: message type message-type, subtype message-subtype, opcode operation-code, length length"
33093,33093,3,"level : 3, log : Number of interface policers configured is exceeding maximum allowed total-count."
42332,42332,3,Oct 21 09:51:00 localhost suricata[1444]: [1:2010494:4] ET SCAN Multiple MySQL Login Failures Possible Brute Force Attempt [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.0.42:3306 -> 192.168.0.160:35188
114336,114336,3,"level : 5, log : Received generic IPC message with incorrect version received-value (expected version expected-value)"
136066,136066,3,"level : 3, log : Reason: reason"
170195,170195,3,juniper
259857,259857,3,"level : 3, log : Unable to update PID file for program-name"
278467,278467,3,Feb 8 16:14:50 localhost sshd[4282]: Exiting on signal 15


## Level1 

In [ ]:
lv_1 = train_df.loc[train_df.level==1] #132517

In [ ]:
lv_1.head(50)

,id,level,full_log
4,4,1,"type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm=""date"" exe=""/usr/bin/date"" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603094402.016:52981): cwd=""/root"" type=PATH msg=audit(1603094402.016:52981): item=0 name=""/etc/ld.so.cache"" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564"
5,5,1,"type=SYSCALL msg=audit(1611889244.855:247124): arch=c000003e syscall=2 success=yes exit=3 a0=7f1c14d535a4 a1=80000 a2=1 a3=7f1c14f594f8 items=1 ppid=100038 pid=100039 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm=""ps"" exe=""/usr/bin/ps"" subj=system_u:system_r:unconfined_service_t:s0 key=""audit-wazuh-r"" type=CWD msg=audit(1611889244.855:247124): cwd=""/"" type=PATH msg=audit(1611889244.855:247124): item=0 name=""/etc/ld.so.cache"" inode=35395307 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611889244.855:247124): proctitle=2F62696E2F7073002D700032353336"
10,10,1,"type=SYSCALL msg=audit(1611895525.367:489785): arch=c000003e syscall=59 success=yes exit=0 a0=14de250 a1=14de360 a2=14dd150 a3=7ffd3604d7a0 items=2 ppid=7383 pid=7384 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm=""ps"" exe=""/usr/bin/ps"" subj=system_u:system_r:unconfined_service_t:s0 key=""audit-wazuh-x"" type=EXECVE msg=audit(1611895525.367:489785): argc=3 a0=""/bin/ps"" a1=""-p"" a2=""2514"" type=CWD msg=audit(1611895525.367:489785): cwd=""/"" type=PATH msg=audit(1611895525.367:489785): item=0 name=""/bin/ps"" inode=101254309 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:bin_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PATH msg=audit(1611895525.367:489785): item=1 name=""/lib64/ld-linux-x86-64.so.2"" inode=80122 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:ld_so_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611895525.367:489785): proctitle=2F62696E2F7073002D700032353134"
15,15,1,"type=SYSCALL msg=audit(1603072847.573:38657): arch=c000003e syscall=2 success=yes exit=8 a0=7fa584948547 a1=80000 a2=1b6 a3=24 items=1 ppid=1 pid=30216 auid=0 uid=0 gid=985 euid=0 suid=0 fsuid=0 egid=985 sgid=985 fsgid=985 tty=(none) ses=1 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603072847.573:38657): cwd=""/"" type=PATH msg=audit(1603072847.573:38657): item=0 name=""/etc/group"" inode=68678689 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:passwd_file_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603072847.573:38657): proctitle=""/var/esild/bin/ossec-syscheckd"""
22,22,1,"type=SYSCALL msg=audit(1603159410.494:164830): arch=c000003e syscall=2 success=yes exit=8 a0=6cbc60 a1=0 a2=1b6 a3=24 items=1 ppid=1 pid=30216 auid=0 uid=0 gid=985 euid=0 suid=0 fsuid=0 egid=985 sgid=985 fsgid=985 tty=(none) ses=1 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603159410.494:164830): cwd=""/"" type=PATH msg=audit(1603159410.494:164830): item=0 name=""/bin/gucharmap"" inode=1140299 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:bin_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(16031

In [ ]:
train_df[train_df['full_log'].str.contains('type=SYSCALL')].value_counts('level')

level
1    116838
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('session opened for user')].value_counts('level')

level
1    4614
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('session closed')].value_counts('level')

level
1    4770
0    2   
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('System Audit')].value_counts('level')

level
1    820
0    17 
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('type=AVC')].value_counts('level')

level
1    369
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Windows Audit')].value_counts('level')

level
1    13
dtype: int64

level
1    369
dtype: int64

In [ ]:
lv_1_1 = lv_1[~lv_1['full_log'].str.contains('Windows Audit')]

In [ ]:
lv_1_1 = lv_1_1[~lv_1_1['full_log'].str.contains('System Audit')]

In [ ]:
lv_1_1.head(50)

,id,level,full_log
88,88,1,Oct 6 16:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/ping -c 3 192.168.0.0
188,188,1,Oct 21 07:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/agent_control -l sed s/Agent Name://
189,189,1,Dec 15 16:52:49 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/agents ; USER=root ; COMMAND=/var/esild/bin/agent_control -i 021 sed s/Agent Name://
246,246,1,Jan 5 08:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/cat /var/esild/etc/ossec.conf
260,260,1,Dec 24 08:00:02 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/ping -c 3 192.168.0.0
262,262,1,Mar 2 22:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/ossec-control status
362,362,1,Nov 28 23:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/ossec-control status
419,419,1,Feb 15 12:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/cat /var/esild/etc/ossec.conf
581,581,1,Sep 27 09:00:02 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/ossec-control status
821,821,1,Jan 12 04:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/agent_control -l sed s/Agent Name://


### Level 0

In [ ]:
lv_0 = train_df.loc[train_df.level==0] #334065

In [ ]:
lv_0.head(50)

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
1,1,0,"Feb 8 16:21:00 localhost logstash: [2021-02-08T16:21:00,548][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[wazuh-alerts-audit-3.x-2021.16.08][3] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-3.x-2021.16.08][3]] containing [25] requests]""})"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
6,6,0,"Jan 22 06:28:59 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-21T21:28:59Z"",""tags"":[""warning"",""stats-collection""],""pid"":4485,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
7,7,0,"Jan 19 09:19:14 localhost logstash: [2021-01-19T09:19:14,483][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>""http://127.0.0.1:9200/"", :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError, :error=>""Elasticsearch Unreachable: [http://127.0.0.1:9200/][Manticore::SocketException] 연결이 거부됨 (Connection refused)""}"
8,8,0,"Jan 20 08:16:43 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T23:16:43Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
9,9,0,"Jan 19 09:26:15 localhost kibana: {""type"":""log"",""@timestamp"":""2021-01-19T00:26:15Z"",""tags"":[""warning"",""monitoring-ui"",""kibana-monitoring""],""pid"":9502,""message"":""Error: No Living connections\n at sen

In [ ]:
lv_0_1 = lv_0[~lv_0['full_log'].str.contains('can not be represented as java')]

In [ ]:
lv_0_1 = lv_0_1[~lv_0_1['full_log'].str.contains('can not be represented as java')]

In [ ]:
lv_0_1.head(50)

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
1,1,0,"Feb 8 16:21:00 localhost logstash: [2021-02-08T16:21:00,548][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[wazuh-alerts-audit-3.x-2021.16.08][3] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-3.x-2021.16.08][3]] containing [25] requests]""})"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
6,6,0,"Jan 22 06:28:59 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-21T21:28:59Z"",""tags"":[""warning"",""stats-collection""],""pid"":4485,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
7,7,0,"Jan 19 09:19:14 localhost logstash: [2021-01-19T09:19:14,483][WARN ][logstash.outputs.elasticsearch] Attempted to resurrect connection to dead ES instance, but got an error. {:url=>""http://127.0.0.1:9200/"", :error_type=>LogStash::Outputs::ElasticSearch::HttpClient::Pool::HostUnreachableError, :error=>""Elasticsearch Unreachable: [http://127.0.0.1:9200/][Manticore::SocketException] 연결이 거부됨 (Connection refused)""}"
8,8,0,"Jan 20 08:16:43 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T23:16:43Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
9,9,0,"Jan 19 09:26:15 localhost kibana: {""type"":""log"",""@timestamp"":""2021-01-19T00:26:15Z"",""tags"":[""warning"",""monitoring-ui"",""kibana-monitoring""],""pid"":9502,""message"":""Error: No Living connections\n at sen

In [ ]:
train_df[train_df['full_log'].str.contains('"type":"error"')].value_counts('level')

level
0    102244
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('[가-힣]')].value_counts('level')

level
0    50027
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('"type"=>"unavailable_shards_exception"')].value_counts('level')

level
0    11501
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('"type":"log"')].value_counts('level')

level
0    67913
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Unable to open file')].value_counts('level')

level
0    2246
1    3   
5    1   
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('intercepted an error:')].value_counts('level')

level
0    11389
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Sequel::DatabaseConnectionError')].value_counts('level')

level
0    11536
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('Unable to connect to database')].value_counts('level')

level
0    11411
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('ET SCAN Suspicious inbound to mySQL port 3306')].value_counts('level')

level
0    17921
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('ConfigurationError')].value_counts('level')

level
0    23
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('CommunicationsException')].value_counts('level')

level
0    21719
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('error:')].value_counts('level')

level
0    23528
1    9    
5    1    
6    1    
dtype: int64

In [ ]:
train_df[train_df['full_log'].str.contains('can not be represented as java')].value_counts('level')

level
0    6032
dtype: int64

In [ ]:
pre_df=pre_df.apply(lambda x: re.sub(r'(\\n)',' ',x))
pre_df=pre_df.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+','',x))
pre_df=pre_df.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
pre_df=pre_df.apply(lambda x: re.sub(r'[0-9]+','<num>',x))

## 레벨별 첫 단어 분포

In [ ]:
first_strings = pd.merge(
    
    train_df.loc[train_df.level==0].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==0'),
    train_df.loc[train_df.level==1].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==1'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings = pd.merge(
    first_strings,
    train_df.loc[train_df.level==2].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==2'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings = pd.merge(
    first_strings,
    train_df.loc[train_df.level==3].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==3'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings = pd.merge(
    first_strings,
    train_df.loc[train_df.level==4].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==4'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings = pd.merge(
    first_strings,
    train_df.loc[train_df.level==5].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==5'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings = pd.merge(
    first_strings,
    train_df.loc[train_df.level==6].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==6'),
    'outer',left_index=True,right_index=True).fillna(0).astype(int)
first_strings = first_strings.sort_values('lv==0',ascending=False) #띄워쓰기를 기준으로 첫번째 단어를 이상치 유/무 기준으로 df생성

In [ ]:
first_strings

,lv==0,lv==1,lv==2,lv==3,lv==4,lv==5,lv==6
Jan,200574,3019,0,3414,1,14,3
Feb,33114,2381,0,44,0,1,4
Dec,24370,1498,0,14,0,0,0
Sep,22392,799,1,1,0,0,0
Oct,20973,2904,0,6,0,0,0
Nov,18980,2646,0,239,9,1,0
Mar,11368,804,0,1,0,0,1
level,1519,736,0,11,0,65,0
error:,459,0,0,0,0,0,0
E:,297,0,0,0,0,0,0


In [ ]:
train_df['full_log'][0]

'Sep 24 10:02:22 localhost kibana: {"type":"error","@timestamp":"2020-09-24T01:02:22Z","tags":["warning","stats-collection"],"pid":6458,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\\n    at process._tickCallback (internal/process/next_tick.js:61:11)"},"message":"No Living connections"}'

In [ ]:
%%time
word_set = set()
for x in lv_2_log.iteritems():
    word_set.update(x[1].split(' '))

print(len(word_set))
' '.join(list(word_set))

36
Wall time: 0 ns


' management of root COMMAND esildbackup : cat number unknown PWD USER apache is html <num> = snapshots The TTY . reached curl between / average var logs bin sudo www XGET and Sep localhost We'

## localhost의 종류 및 특징

In [ ]:
train_local = train_df[train_df['full_log'].str.contains('localhost')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349334 entries, 0 to 472971
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        349334 non-null  int64 
 1   level     349334 non-null  int64 
 2   full_log  349334 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.7+ MB


In [ ]:
train_df.info() #123638은 localhost가 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472972 entries, 0 to 472971
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        472972 non-null  int64 
 1   level     472972 non-null  int64 
 2   full_log  472972 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.8+ MB


In [ ]:
train_df.iloc[472968]

id          472968                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
level       1                                                                                                                                                                                               

In [ ]:
train_local = train_df[train_df['full_log'].str.contains('localhost')].info()

id          472971                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
level       0                                                                                                                                                                                                                                                                                                                                                                                                                                          

type/msg/arch/syscall/success/exit/a0/a1/a2/a3/items/ppid/pid/auid/uid/gid/euid/suid/fsuid/egid/fsgid/tty
ses/comm/exe/subj/key/type/msg/cwd/type/msg/item/name/inode/cap_fi/cap_fe/cap_fver/type/msg/proctitle

type/timestamp/tags/pid/level/error/message/name/